<a href="https://colab.research.google.com/github/LeeSeungYun1020/Machine_Learning/blob/main/PyTorch_Tutorial/04_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 신경망 모델 구성

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


## 클래스 정의

In [4]:
class NeuralNetwork(nn.Module): # nn.Module 상속
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x): # 입력 데이터에 대한 연산 구현
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [20]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [21]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


## 모델 레이어

In [22]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [23]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [24]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [25]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2568, -0.5703, -0.1965, -0.1314,  0.6258, -0.3945, -0.3267, -0.4322,
          0.0963, -0.3938,  0.2595,  0.0992,  0.0396,  0.2334, -0.0479,  0.2588,
          0.1410, -0.2071, -0.0766, -0.1861],
        [-0.0397, -0.0023,  0.1884,  0.1532,  0.4790,  0.0958, -0.0661, -0.4277,
         -0.3190,  0.2603,  0.0294,  0.0190, -0.2338,  0.4770, -0.0737,  0.2089,
          0.1327, -0.2218, -0.3004, -0.0065],
        [ 0.4303,  0.2442,  0.1286, -0.1526,  0.7949, -0.0266, -0.3079, -0.2904,
          0.1377,  0.0688,  0.2126, -0.0905, -0.2558,  0.1807, -0.0222,  0.1117,
         -0.0365, -0.0019, -0.2111, -0.3414]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.2568, 0.0000, 0.0000, 0.0000, 0.6258, 0.0000, 0.0000, 0.0000, 0.0963,
         0.0000, 0.2595, 0.0992, 0.0396, 0.2334, 0.0000, 0.2588, 0.1410, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.1884, 0.1532, 0.4790, 0.0958, 0.0000, 0.0000, 0.0000,
         0.2603, 0.0294, 0.0190, 0.0000, 0.4770, 0.000

In [26]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

tensor([[-0.0178,  0.1531, -0.0355,  0.0648, -0.0986,  0.1745,  0.1654,  0.1598,
         -0.0421,  0.0367],
        [-0.1466,  0.2282, -0.0624, -0.0625, -0.1833,  0.2792,  0.1579,  0.1556,
          0.1245,  0.0188],
        [-0.1903,  0.2723,  0.0202, -0.0644, -0.0259,  0.1620,  0.1184, -0.1197,
          0.0558,  0.0869]], grad_fn=<AddmmBackward>)


In [32]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(softmax)
print(pred_probab)
#p = pred_probab[0] + pred_probab[1] + pred_probab[2]
#print(p / 3)

Softmax(dim=1)
tensor([[0.0925, 0.1097, 0.0908, 0.1004, 0.0853, 0.1121, 0.1110, 0.1104, 0.0902,
         0.0976],
        [0.0811, 0.1180, 0.0883, 0.0883, 0.0782, 0.1242, 0.1100, 0.1098, 0.1064,
         0.0957],
        [0.0794, 0.1261, 0.0980, 0.0901, 0.0936, 0.1130, 0.1081, 0.0852, 0.1016,
         0.1048]], grad_fn=<SoftmaxBackward>)


## 모델 파라미터

In [35]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0142,  0.0124,  0.0089,  ...,  0.0305,  0.0306,  0.0274],
        [ 0.0260, -0.0063, -0.0253,  ...,  0.0078,  0.0270, -0.0022]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0107, -0.0130], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0371, -0.0180, -0.0087,  ...,  0.0183, -0.0321,  0.0162],
        [-0.0412,  0.0363,  0.0299,  ..., -0.0069, -0.0249,  0.0388]],
       device='cuda